In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

!pip install dppy

import torch
import numpy as np 
import networkx as nx
import matplotlib.pyplot as plt

from dppy.finite_dpps import FiniteDPP
from dppy.utils import example_eval_L_linear

1.11.0+cu113
     |████████████████████████████████| 7.9 MB 5.7 MB/s 
     |████████████████████████████████| 3.5 MB 4.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 2.1 MB/s 


In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

# dataset = Planetoid(root='data/Planetoid', name='Citeseer', transform=NormalizeFeatures())
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Has isolated nodes: False
Has self-loops: False
Is undirected: True


Processing...
Done!


# Get the largest connected subgraph 

In [ ]:
from torch_geometric.utils import to_networkx, from_networkx
import networkx as nx

G = to_networkx(data, to_undirected=True)
# print(nx.is_connected(G))
Subnode = max(nx.connected_components(G), key=len)
print(Subnode)
print(len(Subnode))
SubGraph = G.subgraph(Subnode)
diameter = nx.algorithms.distance_measures.diameter(SubGraph)
print(diameter)
SubnodeList = list(Subnode)
print(SubnodeList)

{0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 100, 101, 102, 103, 104, 105, 107, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 185, 186, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 209, 210, 211, 212, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 248, 

In [ ]:
PyTSubGraph = from_networkx(SubGraph)
print(PyTSubGraph)
SubGraph = to_networkx(PyTSubGraph, to_undirected=True)

Data(edge_index=[2, 10138], num_nodes=2485)


In [ ]:
PyTSubGraph.test_mask = data.test_mask[SubnodeList]
PyTSubGraph.train_mask = data.train_mask[SubnodeList]
PyTSubGraph.val_mask = data.val_mask[SubnodeList]
PyTSubGraph.x = data.x[SubnodeList]
PyTSubGraph.y = data.y[SubnodeList]
print(PyTSubGraph)

Data(edge_index=[2, 10138], num_nodes=2485, test_mask=[2485], train_mask=[2485], val_mask=[2485], x=[2485, 1433], y=[2485])


In [ ]:
num_nodes = PyTSubGraph.num_nodes
edge_index = PyTSubGraph.edge_index
edge_value = torch.ones(edge_index.size(1), device=edge_index.device)
adjdense = torch.sparse.FloatTensor(edge_index, edge_value, torch.Size([num_nodes,num_nodes])).to_dense()
featuresNum = PyTSubGraph.x.shape[1]
print(adjdense.shape)

torch.Size([2485, 2485])


# Shortest Path for node candidates Section

In [ ]:
from networkx import all_pairs_shortest_path
import time
start = time.time()
SPathDict = dict(nx.all_pairs_shortest_path(SubGraph))
end = time.time() 
print(end-start)

17.21773600578308


In [ ]:
from collections import defaultdict
import random

def GetCandidates(SPathDict,totalnum,gap):
  candidates = defaultdict(list)
  SelNodeList = [] 
  for i in range(PyTSubGraph.x.shape[0]):
    centralNode = []
    SortedNode = sorted(SPathDict[i], key=lambda k: len(SPathDict[i][k]), reverse=True)
    '''
    1, Based on all shortest paths of each node, 
      *get the list consisting of the last point on the path 
      that 1)not Longest 20, 2)not first-order nearest neighbors
      *get length of every shortest path(Sorted)
    '''
    Lenlist = []     # length of every shortest path
    nlist = []      # The last node of the shortest path 
    for node in reversed(SortedNode):
      # print(SPathDict[0][node])
      templen = len(SPathDict[i][node])
      if templen > 2: # Exclusion of first-order nearest neighbors
        nlist.append(node)
        Lenlist.append(templen)
    '''
    2, *Select a random Node at each length, delete this node in /Leftlist/
      *Randomly select all remaining points to get a total of 20 Nodes
    '''
    selList = []        # select nodes list
    maxlen = Lenlist[-1]     # length of the longest shortest path
    Lenlist = np.array(Lenlist)
    Leftlist = nlist.copy()
    for l in range(3,3+totalnum*gap,gap):
      llist = list(np.where(Lenlist==l)[0]) #index of certain length
      centrIndex = int(random.sample(llist, 1)[0])
      selList.append(nlist[centrIndex]) 
    candidates[i].extend(selList)
  return (candidates)

In [ ]:
print(GetCandidates(SPathDict,5,1))

defaultdict(<class 'list'>, {0: [1068, 1719, 1052, 1224, 64], 1: [1846, 1862, 263, 670, 1972], 2: [417, 2074, 931, 2037, 2006], 3: [539, 1585, 1100, 991, 1287], 4: [422, 1627, 1576, 1661, 2027], 5: [1912, 1460, 1071, 747, 1494], 6: [915, 1030, 1610, 1131, 2230], 7: [2276, 795, 1576, 2412, 2127], 8: [271, 1733, 2336, 2130, 2422], 9: [1995, 1385, 498, 1197, 2424], 10: [1721, 2157, 1243, 1503, 1461], 11: [440, 1731, 274, 2335, 2067], 12: [355, 1725, 35, 176, 607], 13: [528, 1614, 513, 1181, 345], 14: [1543, 1213, 650, 1943, 1604], 15: [1642, 1456, 744, 704, 2051], 16: [1418, 220, 1250, 700, 970], 17: [1296, 922, 980, 1848, 2121], 18: [1780, 184, 245, 1914, 865], 19: [1827, 1341, 2111, 2223, 30], 20: [1737, 1858, 1362, 2186, 1278], 21: [745, 886, 463, 822, 398], 22: [1887, 178, 387, 1488, 1558], 23: [917, 1259, 975, 1646, 1796], 24: [2241, 738, 1648, 1333, 1658], 25: [914, 2070, 938, 1840, 37], 26: [1402, 1540, 1754, 1292, 1673], 27: [2218, 2190, 1408, 1201, 1460], 28: [559, 2381, 98, 1239

# Community Section

In [ ]:
from networkx.algorithms.community.label_propagation import label_propagation_communities
LabelComu = label_propagation_communities(SubGraph)

In [ ]:
def GetCommunityMatrix(label_community,num_nodes):
  lcomuMatrix = torch.zeros(len(label_community), num_nodes)
  com = 0 
  for c in label_community:
    for ele in c:
      lcomuMatrix[com][ele] = 1
    com = com+1
  return lcomuMatrix

In [ ]:
# Get community matrix / Every node belong to different community
LComuMatrix = GetCommunityMatrix(LabelComu,num_nodes) 
print(LComuMatrix.shape)
print(LComuMatrix.sum(1)[0:10])

torch.Size([373, 2485])
tensor([ 7.,  2.,  2., 11.,  5., 15.,  5.,  4.,  2.,  9.])


In [ ]:
def GetComuFeatureMatrix(label_community,featuresNum,x):
  temp = 0
  n = 0
  ComuFeatureM = torch.zeros(len(label_community), featuresNum)
  for c in lst_l:
    for ele in c:
      temp= temp + x[ele]
    ComuFeatureM[n] = temp / len(c)
    n = n+1
  return ComuFeatureM

# GCN Section

In [ ]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_scatter import gather_csr, scatter
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3-5: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]
        # edge_index has shape [2, E]

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)  # [N, ]
        deg_inv_sqrt = deg.pow(-0.5)   # [N, ]
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j
            
    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from dppy.finite_dpps import FiniteDPP
from dppy.utils import example_eval_L_linear
import time

class D2GCN(torch.nn.Module):
    def __init__(self, hidden_channels,adjdense,num_layers,num_nodes,LabelComu,LComuMatrix):
        super(D2GCN, self).__init__()
        # torch.manual_seed(12345)

        self.num_layers = num_layers
        self.CONVs = torch.nn.ModuleList()
        self.DPPCONVs = torch.nn.ModuleList()
        self.NegRates = nn.ParameterList()
        for layer in range(self.num_layers-1):
            if layer == 0:
                self.CONVs.append(GCNConv(dataset.num_features, hidden_channels))
                self.DPPCONVs.append(GCNConv(dataset.num_features, hidden_channels))
                self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))
            else:
                self.CONVs.append(GCNConv(hidden_channels, hidden_channels))
                self.DPPCONVs.append(GCNConv(hidden_channels, hidden_channels))
                self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))
        self.CONVs.append(GCNConv(hidden_channels, dataset.num_classes))
        self.DPPCONVs.append(GCNConv(hidden_channels, dataset.num_classes))
        self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))

        self.LabelComu = LabelComu
        self.LComuMatrix = LComuMatrix
        self.adjdense = adjdense
        self.num_nodes = num_nodes
        # According to experimental experience, setting NegRate to 1~3 can achieve better results
        self.NegRate = 1


    def forward(self, x, edge_index,candidates,DPPedge_index_list,train_neg_rate):
        for layer in range(self.num_layers):
          ComuFeatureM = self.GetComuFeatureMatrix(x)
          x_temp = x.clone()
          posi_x = self.CONVs[layer](x, edge_index)
          if len(DPPedge_index_list) < 1:
            DppMatrix = self.GetDppMatrix(x_temp,candidates,ComuFeatureM)
            #change dense back to sparse
            Dppedge_index = torch.nonzero(DppMatrix).t()
            DPPedge_index_list.append(Dppedge_index)
          else:
            Dppedge_index = DPPedge_index_list[0]

          nega_x = self.DPPCONVs[layer](x_temp, Dppedge_index)
          if train_neg_rate: #NegRate is a trainable parameter
            x = posi_x - self.NegRates[layer] * nega_x
          else:
            x = posi_x - self.NegRate * nega_x

          if layer < (self.num_layers-1):
            x = x.relu()
            x = F.dropout(x, p=0.5, training=self.training)
        return x,DPPedge_index_list
        
    def GetComuFeatureMatrix(self,x):
        temp = 0
        n = 0
        ComuFeatureM = torch.zeros(len(self.LabelComu), x.shape[1])
        for c in self.LabelComu:
          for ele in c:
            temp= temp + x[ele]
          ComuFeatureM[n] = temp / len(c)
          n = n+1
        return ComuFeatureM

    def GetDppMatrix(self,x,candidates,ComuFeatureM):
        DppMatrix = torch.zeros((self.num_nodes,self.num_nodes))
        for node in candidates:
          AdjList = []
          #Get community index of central node "node"
          cumIndex = torch.nonzero(self.LComuMatrix[:,node]).reshape(-1)
          #Get feature of community of "node"
          CNodeFeature = ComuFeatureM[cumIndex.item()]
          for CentralIndex in candidates[node]:
              AjdIndex = torch.nonzero(self.adjdense[CentralIndex]) # get adj node index of central node
              AjdIndex = torch.reshape(AjdIndex, (-1,))
              AjdIndex = AjdIndex.tolist()
              AdjList.extend(AjdIndex)
          sampleAmount = int(adjdense[node].sum().item()) + 1
          if sampleAmount > len(AdjList):
            sampleAmount = len(AdjList)
          DppIndex = self.Dppsample(x, AdjList, sampleAmount,ComuFeatureM,CNodeFeature)
          DppMatrix[node][DppIndex] = 1
        return DppMatrix
    
    def Dppsample(self,x, AjdIndex,sampleAmount,ComuFeatureM,CNodeFeature):
        #index list of cummunity of every node
        cumiList = []
        for node in AjdIndex:
          cumIndex = torch.nonzero(self.LComuMatrix[:,node]).reshape(-1)
          cumiList.append(cumIndex.item())
        Index = self.Rek_dpp_kernel(x[AjdIndex],ComuFeatureM[cumiList],CNodeFeature,sampleAmount)
        AjdIndex = np.array(AjdIndex)
        DppIndex = AjdIndex[Index]
        return DppIndex

    def Rek_dpp_kernel(self, NodeWeight,ComuWeight,CNodeFeature,k_number):
        NodecosSim = self.CosSimilarity(NodeWeight,NodeWeight)
        ComucosSim = self.CosSimilarity(NodeWeight,ComuWeight)
        
        cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
        NodeFeatureCos = cos(CNodeFeature,(NodeWeight.sum(0)/NodeWeight.shape[0]))

        CNodeFeMatrix = torch.ones_like(ComuWeight)
        CNodeFeMatrix = CNodeFeMatrix * CNodeFeature
        QijMatrix =  NodeFeatureCos * self.CosSimilarity(CNodeFeMatrix,ComuWeight) 

        Lmatrix = QijMatrix*torch.mm(ComucosSim, ComucosSim.transpose(0,1)) * torch.exp(NodecosSim-1)* QijMatrix.transpose(0,1)
        Lmatrix = Lmatrix.detach().numpy() + 0.01*np.eye(Lmatrix.shape[0])
        Lmatrix = np.float64(Lmatrix)
        DPP = FiniteDPP('likelihood', **{'L': Lmatrix})
        
        DPP.flush_samples()
        DPP.sample_exact_k_dpp(size=k_number)
        values = np.squeeze(DPP.list_of_samples)
        
        return values
    
    def CosSimilarity(self,a,b):
        AnormTemp = a.norm(dim=1)[:, None]
        AoneTemp = torch.ones_like(AnormTemp)
        AnormTemp = torch.where(AnormTemp == 0,AoneTemp,AnormTemp)
        Aweight_norm = a / AnormTemp
        BnormTemp = b.norm(dim=1)[:, None]
        BoneTemp = torch.ones_like(AnormTemp)
        BnormTemp = torch.where(BnormTemp == 0,BoneTemp,BnormTemp)
        Bweight_norm = b / BnormTemp
        cosSim = torch.mm(Aweight_norm, Bweight_norm.transpose(0,1))
        return cosSim

In [ ]:
def MAD(x):
  x_norm = x / x.norm(dim=1)[:, None]
  dist = 1-torch.mm(x_norm, x_norm.transpose(0,1))
  one1 = torch.ones_like(dist)
  zreo1 = torch.zeros_like(dist)
  dist_1 = torch.where(dist > 0, one1, zreo1)
  D = dist.sum(0)/dist_1.sum(0)
  one = torch.ones_like(D)
  zero = torch.zeros_like(D)
  D_1 = torch.where(D > 0, one, zero)
  Mad = D.sum()/D_1.sum()
  return Mad.detach().numpy()


In [ ]:
import sys

def test(candidates,DPPedge_index_list,train_neg_rate):
      model.eval()
      out,_ = model(PyTSubGraph.x, edge_index,candidates,DPPedge_index_list,train_neg_rate)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[PyTSubGraph.test_mask] == PyTSubGraph.y[PyTSubGraph.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(PyTSubGraph.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

#----------------
# Set training parameters
num_layers = 4
Runs = 11
Epochs = 201
# sel_num = 10
#--------------------
#Set whether NegRate is a trainable parameter
#if False:
#   NegRate = 1
train_neg_rate = True
#-----------------

for num in range(1,Runs):
  model = D2GCN(hidden_channels=16,
          adjdense=adjdense,
          num_layers=num_layers,
          num_nodes=num_nodes,
          LabelComu=LabelComu,
          LComuMatrix=LComuMatrix)


  print("num_layers",num_layers)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  criterion = torch.nn.CrossEntropyLoss()
  begin = 0
  if num == 1 :
    print(model)
    # begin = 25

    # PATH = "/content/drive/My Drive/DBLP-D2GCN-Layer4-Epoch"+str(begin)+".pth"
    # checkpoint = torch.load(PATH)
    # model.load_state_dict(checkpoint['model_state_dict'])
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

  for epoch in range(begin+1, Epochs):
      start = time.time()
      DPPedge_index_list = []
      model.train()
      optimizer.zero_grad()  # Clear gradients.

      #Get candidates
      candidates = GetCandidates(SPathDict,5,1)

      out,DPPedge_index_list = model(PyTSubGraph.x, edge_index,candidates,DPPedge_index_list,train_neg_rate)  
      loss = criterion(out[PyTSubGraph.train_mask], PyTSubGraph.y[PyTSubGraph.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()
      end = time.time()
      if epoch == 1 or epoch == begin+1:
          print("---------------")
          print("DPPedge_index_list",DPPedge_index_list[0].shape)
          print("Epoch time:",end-start)
          # print("walks_num",walks_num)
          # print("sel_num",sel_num)
          print("---------------")
      if epoch % 5 == 0 or epoch >100:
          test_acc = test(candidates,DPPedge_index_list,train_neg_rate)
          print(f'num: {num:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f},Test Accuracy: {test_acc:.4f},MAD: {MAD(out):.4f}')

num_layers 4
D2GCN(
  (CONVs): ModuleList(
    (0): GCNConv()
    (1): GCNConv()
    (2): GCNConv()
    (3): GCNConv()
  )
  (DPPCONVs): ModuleList(
    (0): GCNConv()
    (1): GCNConv()
    (2): GCNConv()
    (3): GCNConv()
  )
  (NegRates): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 1]
      (1): Parameter containing: [torch.FloatTensor of size 1]
      (2): Parameter containing: [torch.FloatTensor of size 1]
      (3): Parameter containing: [torch.FloatTensor of size 1]
  )
)
---------------
DPPedge_index_list torch.Size([2, 12052])
Epoch time: 21.65669298171997
---------------
num: 001, Epoch: 005, Loss: 1.9062,Test Accuracy: 0.1377,MAD: 0.2048
num: 001, Epoch: 010, Loss: 1.8816,Test Accuracy: 0.1137,MAD: 0.2364
num: 001, Epoch: 015, Loss: 1.8574,Test Accuracy: 0.1628,MAD: 0.3013
num: 001, Epoch: 020, Loss: 1.8385,Test Accuracy: 0.1792,MAD: 0.4590
num: 001, Epoch: 025, Loss: 1.7407,Test Accuracy: 0.2732,MAD: 0.6624
num: 001, Epoch: 030, Loss: 1.6613,